In [1]:
import CSV, DataFrames
import Random

t = Matrix( CSV.read("surface_512x512.txt",DataFrames.DataFrame, header=false) )
;

ModuleNotFoundError: No module named 'CSV'

In [ ]:
import Plots
Plots.gr(fmt = :png)

In [ ]:
Plots.scatter3d(t[:,1],t[:,2],t[:,3],
    size=(900,700),legend=false, 
    ms=1.4,mscolor=:blue,
    title = "Energia-szintek"
)
# NAAAGYON lassuuuu!!!

In [ ]:
n_sq = Int(round(sqrt(size(t,1))))
z = reshape(t[:,3],(n_sq,n_sq))
ind_obst = findall(t[:,4].==1)
Plots.gr()
p = Plots.contour(z, colorbar=false,size=(800,500))
Plots.scatter!(p,t[ind_obst,1],t[ind_obst,2],marker=:rect,ms=1.3,legend=false,
    title = "Pálya akadályokkal"
)

In [ ]:
# getting the obstacles

dd = Dict([ (i,j) => b for (i,j,k,b) in eachrow(t) if b==1]...)

@show length(dd)

In [ ]:
# declaring the parameter for the wall:
s_neigh = 14

In [ ]:
# removing within NEIGH
for (i_key,_) in dd
    # @show i_key
    for (j_key,_) in dd
        if (i_key != j_key) && maximum( abs.(i_key .- j_key) ) < s_neigh
            delete!(dd,j_key)
        end
    end
end
@show length(dd)

In [ ]:
# displaying again the obstacles
p = Plots.contour(z, colorbar=false,size=(800,500))
r_x = hcat([[z[1],z[2]] for (z,_) in dd]...)
p = Plots.scatter!(p, r_x[1,:],r_x[2,:],legend=false,ms=2,marker=:rect,
    title = "Ritkított akadályok"
)

In [ ]:
# adding walls
n_brick = 1.5 * s_neigh
dd2 = Dict()
for (pos,_) in dd
    dir = sign(randn())
    e = randn()>0 ? (1,0) : (0,1)
    for i=0:n_brick
        p2 = pos .+ i .* e
        if all(p2 .< n_sq)
            merge!(dd2, Dict(p2 => 1) )
        end
    end
end

In [ ]:
p = Plots.contour(z, colorbar=false,size=(800,500))
r_x = hcat([[z[1],z[2]] for (z,_) in dd2]...)
p = Plots.scatter!(p, r_x[1,:],r_x[2,:],legend=false,marker=:rect,ms=1.3,
    title = "Hosszabbított akadályok"
)
# Plots.savefig("contour_$(n_sq).pdf")

In [ ]:
# saving into a CSV
import DelimitedFiles

open("large_512x512_n.csv","w") do io
    for jj = 1:n_sq
        for ii = 1:n_sq 
            write(io,"$(ii-1) $(jj-1) $(z[ii,jj]) $((haskey(dd2,(ii,jj)) ? 1 : 0))\n")
        end
    end
end
